In [1]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import numpy as np
import warnings
warnings.filterwarnings('ignore')

/home/sanidhya/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_df = pd.read_csv(r'Training.csv')
test_df = pd.read_csv(r'Testing.csv')

In [3]:
df = pd.concat([train_df,test_df],axis=0)
df.shape

(4962, 2)

In [4]:
df.reset_index(inplace=True)

In [5]:
df.columns

Index(['index', 'symptoms', 'prognosis'], dtype='object')

In [6]:
le = LabelEncoder()
# df['prognosis'] = le.fit_transform(df['prognosis'])
temp = le.fit_transform(df['prognosis'])
labels = dict()
for i in range(len(temp)):
    labels[df['prognosis'][i]] = temp[i]

In [7]:
keys = []
val = []
for i in labels.keys():
    keys.append(i)
    val.append(labels[i])
labels_Data = {
    'keys': keys,
    'val' : val
}
labels_Data = pd.DataFrame(labels_Data)
labels_Data.to_csv('Labels_data.csv',index=False)

In [8]:
df['prognosis'] = temp

In [9]:
# Set the number of labels in the dataset
num_labels = len(df['prognosis'].unique())

In [10]:
# Load the pre-trained BERT model and its tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
# Define the optimizer and the loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [12]:
X = [str(i) for i in df['symptoms'].values]
y = np.array(df['prognosis'])

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.2)

In [14]:
# Tokenize the input texts
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128)
eval_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128)

In [15]:
# Convert the input data to PyTorch tensors
train_inputs = torch.tensor(train_encodings['input_ids'])
train_labels = torch.tensor(y_train)
train_masks = torch.tensor(train_encodings['attention_mask'])

In [16]:
eval_inputs = torch.tensor(eval_encodings['input_ids'])
eval_labels = torch.tensor(y_test)
eval_masks = torch.tensor(eval_encodings['attention_mask'])

In [17]:
# Train the model
for epoch in range(5):
    model.train()

    optimizer.zero_grad()

    outputs = model(train_inputs, attention_mask=train_masks, labels=train_labels)
    loss = loss_fn(outputs.logits, train_labels)

    loss.backward()
    optimizer.step()

    print('Epoch:', epoch, 'Loss:', loss.item())

: 

: 

In [ ]:
# Save the trained model
model.save_pretrained('model')